In [ ]:

# %pip install SQLAlchemy==2.0.19 pandas==2.2.2


In [ ]:

import os
import socket
import sys

# функция, которая проверяет отсутствие имен файлов из кортежа tup_fns в содержимом папки path_
def check_missing_files(tup_fns, path_):
    # Получаем список файлов в указанной папке
    files_in_directory = os.listdir(path_)
    
    # Создаём список файлов, которых нет в папке
    missing_files = [file for file in tup_fns if file not in files_in_directory]
    
    # Проверка и вывод результата
    if missing_files:
        print(f"Следующие файлы отсутствуют в папке '{path_}': {', '.join(missing_files)}")
        sys.exit()
    else:
        print(f"Все необходимые для работы программы служебные файлы присутствуют в папке '{path_}'.")
    
    return missing_files

# кортеж с именами необходимых для работы программы служебных файлов
tup_fns = (
    'user_functions.py',
    'paths_and_constants.py',
    'dict_sql_queries.py',
    'get_dicts_v_06.py',
)


# получаем имя компьютера
hostname = socket.gethostname()

# если программа выполняется на сервере
if hostname == '304-007':
    # указываем путь к общим документам
    path_ = r'C:\Users\Public\dictionaries'
# иначе (если программа выполняется на своём рабочем ПК)
else:
    # указываем путь к документам активного пользователя
    path_ = os.path.expanduser('~\\Documents\\')

# проверяем отсутствие имен файлов из кортежа tup_fns в содержимом папки path_
check_missing_files(tup_fns, path_)

# добавляем путь к системному пути поиска модулей (путь к папке, в которой Python ищет модули)
sys.path.append(path_)

# далее импорт пользовательских библиотек идёт с пути `path_`
try:
    import get_dicts_v_06
except:
    print(f'\nДоп. модули, необходиимые для работы кода, НЕ НАЙДЕНЫ по адресу "{path_}"')

# использование пользовательской функции для получения словарей из SQL
# можно ввести номера словарей через пробел в скобки, если словари заранее известны
dict_dicts, tables = get_dicts_v_06.main('1 2')  #  <------------------------------------------------------------------------------------ ВВОД!

# Цикл по индексам от 0 до длины списка tables
for i in range(len(tables)):
    try:
        # Используем exec() для динамического создания переменной с именем таблицы (tables[i][0]) 
        # и присваивания ей первого элемента (таблицы) из соответствующего словаря dict_dicts
        exec(f"{tables[i][0]} = dict_dicts[tables[i][0]][0]")
        # Вывод информации о таблице, включая её название, заголовок, и дату загрузки в БД
        print(f"\nТаблицей '{tables[i][0]}' можно пользоваться. "
              f"Её название: {dict_dicts[tables[i][0]][1]} "
              f"Она была загружена в БД {dict_dicts[tables[i][0]][2]}")
        # Отображаем первые две строки таблицы с помощью функции display()
        display(dict_dicts[tables[i][0]][0].head(2))
    except:
        # print(f"\nСкорее всего, таблицы '{tables[i][0]}' ещё не существует в БД")
        # Если происходит ошибка (например, если таблицы ещё нет в БД), просто пропускаем и ничего не выводим.
        pass
